In [2]:
from dataclasses import dataclass
import einops
import math
import torch as t
from torch import nn
import torch.nn. functional as F
from torchtyping import TensorType
import transformers
from typing import Optional

import gpt_tests

In [3]:
class UniAttention(nn.Module):
    def __init__(self, hidden_size: int, num_heads: int):
        super(UniAttention, self).__init__()
        self.attention = nn.Linear(hidden_size, hidden_size * 3)
        self.output = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        x = self.attention(x)
        a, b, c = t.split(x, 2, dim=-1)
        a = self.rearrange_heads(a, self.num_heads)
        b = self.rearrange_heads(b, self.num_heads)
        c = self.rearrange_heads(c, self.num_heads)
        attn_score = t.einsum('b m h p, b n h p -> b h n m', a, b)
        attn_score = einops.rearrange(attn_score, 'b h n m -> (b h) n m')
        masked = t.triu(attn_score)
        masked_arranged = einops.rearrange(masked, '(b h) n m -> b h n m')
        attn_pat = t.softmax(masked_arranged, dim=2)
        abc = t.einsum('b h n m, b n h p -> b m h p', attn_pat, c)
        abc_arranged = einops.rearrange(abc, 'b m h p -> b m (h p)')
        output = self.output(abc_arranged)
        return output
    @staticmethod
    def rearrange_heads(t, num_heads):
        return einops.rearrange(t, 'b n (p h) -> b n h p', h=num_heads)

gpt_tests.test_unidirectional_attn(UniAttention)

ValueError: too many values to unpack (expected 3)